In [ ]:
# Download entire csv.file: https://www.kaggle.com/bulter22/airline-data?select=airline.csv.shuffle

In [ ]:
# Sort the csv file on Years, and distribute them to individual csv-files

import csv

file = open('airline.csv.shuffle')
csvreader = csv.reader(file)

header = []
header = next(csvreader)
header

with open('airline.csv.shuffle') as fin:    
    #csvin = csv.DictReader(fin)
    csvin = csv.DictReader(fin)
    # Category -> open file lookup
    outputs = {}
    for row in csvin:
        cat = row['Year']
        # Open a new file and write the header
        if cat not in outputs:
            fout = open('{}.csv'.format(cat), 'w')
            dw = csv.DictWriter(fout, fieldnames=csvin.fieldnames)
            dw.writeheader()
            outputs[cat] = fout, dw
        # Always write the row
        outputs[cat][1].writerow(row)
    # Close all the files
    for fout, _ in outputs.values():
        fout.close()

In [2]:
import csv
import pandas as pd

In [3]:
df_2008 = pd.read_csv("2008.csv")
df_2008.head()

,ActualElapsedTime,AirTime,ArrDelay,ArrTime,CRSArrTime,CRSDepTime,CRSElapsedTime,CancellationCode,Cancelled,CarrierDelay,...,Month,NASDelay,Origin,SecurityDelay,TailNum,TaxiIn,TaxiOut,UniqueCarrier,WeatherDelay,Year
0,61.0,40.0,-3.0,1537.0,1540,1440,60.0,NaN,0,NaN,...,7,NaN,DFW,NaN,N286AE,7.0,14.0,MQ,NaN,2008
1,85.0,48.0,13.0,623.0,610,500,70.0,NaN,0,NaN,...,1,NaN,GFK,NaN,89709E,10.0,27.0,9E,NaN,2008
2,189.0,175.0,-22.0,433.0,455,2325,210.0,NaN,0,NaN,...,8,NaN,LAS,NaN,N173AT,5.0,9.0,FL,NaN,2008
3,158.0,140.0,-4.0,1434.0,1438,1200,158.0,NaN,0,NaN,...,12,NaN,BUF,NaN,N267AT,8.0,10.0,FL,NaN,2008
4,71.0,42.0,2.0,1820.0,1818,1716,62.0,NaN,0,NaN,...,10,NaN,LFT,NaN,N13968,5.0,24.0,XE,NaN,2008


In [4]:
# Remove the flights that have been cancelled (since these do not have an ArrTime)
df_2008 = df_2008[df_2008['Cancelled'] == 0]
df_2008.dtypes

ActualElapsedTime    float64
AirTime              float64
ArrDelay             float64
ArrTime              float64
CRSArrTime             int64
CRSDepTime             int64
CRSElapsedTime       float64
CancellationCode      object
Cancelled              int64
CarrierDelay         float64
DayOfWeek              int64
DayofMonth             int64
DepDelay             float64
DepTime              float64
Dest                  object
Distance               int64
Diverted               int64
FlightNum              int64
LateAircraftDelay    float64
Month                  int64
NASDelay             float64
Origin                object
SecurityDelay        float64
TailNum               object
TaxiIn               float64
TaxiOut              float64
UniqueCarrier         object
WeatherDelay         float64
Year                   int64
dtype: object

In [5]:
# Remove the flights that do not have an ArrTime
df_2008.dropna(subset = ["ArrTime"], inplace=True)

In [6]:
# Sort the values on Month, DayofMonth
df_2008.sort_values(["Month","DayofMonth", "ArrTime"],axis=0, ascending=True,inplace=True,ignore_index=True, na_position='first')
#ignore_index: this changes the indices so that it again starts at 0 and ends at the last one
print("\nSorted CSV file (according to multiple columns) = \n", df_2008)


Sorted CSV file (according to multiple columns) = 
          ActualElapsedTime  AirTime  ArrDelay  ArrTime  CRSArrTime  \
0                    315.0    294.0       6.0      1.0        2355   
1                    149.0    133.0      14.0      1.0        2347   
2                    202.0    184.0      31.0      1.0        2330   
3                     73.0     57.0       6.0      1.0        2355   
4                    297.0    268.0      11.0      1.0        2350   
...                    ...      ...       ...      ...         ...   
6858074              159.0    141.0      35.0   2400.0        2325   
6858075              236.0    213.0      26.0   2400.0        2334   
6858076              156.0    133.0       5.0   2400.0        2355   
6858077              203.0    177.0       7.0   2400.0        2353   
6858078              203.0    139.0      48.0   2400.0        2312   

         CRSDepTime  CRSElapsedTime CancellationCode  Cancelled  CarrierDelay  \
0              1545      

In [7]:
# Drop the columns that we do not need to use for Data_streaming and Batch_streaming
df_2008 = df_2008.drop(columns=['SecurityDelay', 'TailNum', 'WeatherDelay', 'TaxiIn', 'TaxiOut', 'ActualElapsedTime', 'AirTime',
                     'CRSArrTime', 'CRSDepTime', 'CRSElapsedTime', 'Cancelled', 'CarrierDelay',
                                'Distance', 'Diverted', 'FlightNum', 'LateAircraftDelay', 'CancellationCode'])

# CancellationCode and Cancelled can be removed since we do not look at Cancelled flights

In [ ]:
df_2008.head()

In [ ]:
# Building a Unix Timestamp

In [8]:
# Build a function that splits ArrTime into hours and minutes

def split_ArrTime(time):
    time_in_digits = [int(d) for d in str(int(time))]
    if len(time_in_digits) == 1:
        hour = 0
        minutes = time_in_digits[0]
        return hour, minutes
    elif len(time_in_digits) == 2:
        hour = 0
        minutes = time_in_digits
        return hour, convert_list(minutes)
    elif len(time_in_digits) == 3:
        hour = time_in_digits[0]
        minutes = time_in_digits[1:]
        return hour, convert_list(minutes)
    else:
        hour = time_in_digits[0:2]
        minutes = time_in_digits[2:]
        if convert_list(hour) == 24:
            hour = 0
        else: 
            hour = convert_list(hour)
        return hour, convert_list(minutes)

In [9]:
# Building a function that converts the list (function used in split_ArrTime)

def convert_list(lst):
    s = [str(i) for i in lst]
    res = int("".join(s)) 
    return(res)

In [10]:
ArrTime = df_2008["ArrTime"]

time = [split_ArrTime(time) for time in ArrTime]
year = df_2008['Year']
month = df_2008['Month']
day = df_2008['DayofMonth']
hour = [t[0] for t in time] 
minute = [t[1] for t in time]
readabletime = list(zip(year,month,day,hour,minute))
readabletime

[(2008, 1, 1, 0, 1),
 (2008, 1, 1, 0, 1),
 (2008, 1, 1, 0, 1),
 (2008, 1, 1, 0, 1),
 (2008, 1, 1, 0, 1),
 (2008, 1, 1, 0, 1),
 (2008, 1, 1, 0, 1),
 (2008, 1, 1, 0, 1),
 (2008, 1, 1, 0, 1),
 (2008, 1, 1, 0, 2),
 (2008, 1, 1, 0, 2),
 (2008, 1, 1, 0, 2),
 (2008, 1, 1, 0, 2),
 (2008, 1, 1, 0, 2),
 (2008, 1, 1, 0, 2),
 (2008, 1, 1, 0, 2),
 (2008, 1, 1, 0, 2),
 (2008, 1, 1, 0, 2),
 (2008, 1, 1, 0, 2),
 (2008, 1, 1, 0, 2),
 (2008, 1, 1, 0, 2),
 (2008, 1, 1, 0, 3),
 (2008, 1, 1, 0, 3),
 (2008, 1, 1, 0, 3),
 (2008, 1, 1, 0, 3),
 (2008, 1, 1, 0, 3),
 (2008, 1, 1, 0, 3),
 (2008, 1, 1, 0, 4),
 (2008, 1, 1, 0, 4),
 (2008, 1, 1, 0, 4),
 (2008, 1, 1, 0, 4),
 (2008, 1, 1, 0, 4),
 (2008, 1, 1, 0, 4),
 (2008, 1, 1, 0, 4),
 (2008, 1, 1, 0, 4),
 (2008, 1, 1, 0, 4),
 (2008, 1, 1, 0, 5),
 (2008, 1, 1, 0, 5),
 (2008, 1, 1, 0, 5),
 (2008, 1, 1, 0, 5),
 (2008, 1, 1, 0, 5),
 (2008, 1, 1, 0, 5),
 (2008, 1, 1, 0, 5),
 (2008, 1, 1, 0, 5),
 (2008, 1, 1, 0, 5),
 (2008, 1, 1, 0, 5),
 (2008, 1, 1, 0, 5),
 (2008, 1, 1,

In [11]:
import datetime
import pytz
timestamp = [int(datetime.datetime(year = time[0], month = time[1], day = time[2], hour = time[3], minute = time[4], tzinfo=pytz.utc).timestamp()) for time in readabletime]
timestamp


[1199145660,
 1199145660,
 1199145660,
 1199145660,
 1199145660,
 1199145660,
 1199145660,
 1199145660,
 1199145660,
 1199145720,
 1199145720,
 1199145720,
 1199145720,
 1199145720,
 1199145720,
 1199145720,
 1199145720,
 1199145720,
 1199145720,
 1199145720,
 1199145720,
 1199145780,
 1199145780,
 1199145780,
 1199145780,
 1199145780,
 1199145780,
 1199145840,
 1199145840,
 1199145840,
 1199145840,
 1199145840,
 1199145840,
 1199145840,
 1199145840,
 1199145840,
 1199145900,
 1199145900,
 1199145900,
 1199145900,
 1199145900,
 1199145900,
 1199145900,
 1199145900,
 1199145900,
 1199145900,
 1199145900,
 1199145900,
 1199145900,
 1199145900,
 1199145960,
 1199145960,
 1199145960,
 1199145960,
 1199145960,
 1199145960,
 1199145960,
 1199145960,
 1199145960,
 1199145960,
 1199146020,
 1199146020,
 1199146020,
 1199146020,
 1199146020,
 1199146020,
 1199146020,
 1199146020,
 1199146020,
 1199146020,
 1199146080,
 1199146080,
 1199146080,
 1199146080,
 1199146080,
 1199146080,
 1199146080,

In [12]:
df_2008['Timestamp'] = timestamp
df_2008

,ArrDelay,ArrTime,DayOfWeek,DayofMonth,DepDelay,DepTime,Dest,Month,NASDelay,Origin,UniqueCarrier,Year,Timestamp
0,6.0,1.0,2,1,1.0,1546.0,EWR,1,NaN,SFO,CO,2008,1199145660
1,14.0,1.0,2,1,-3.0,2132.0,MCO,1,NaN,IAD,UA,2008,1199145660
2,31.0,1.0,2,1,14.0,2039.0,MIA,1,17.0,LGA,AA,2008,1199145660
3,6.0,1.0,2,1,8.0,2148.0,ICT,1,NaN,DEN,UA,2008,1199145660
4,11.0,1.0,2,1,4.0,1604.0,PVD,1,NaN,LAS,WN,2008,1199145660
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6858074,35.0,2400.0,3,31,51.0,2121.0,MIA,12,0.0,PHL,US,2008,1230681600
6858075,26.0,2400.0,3,31,9.0,2204.0,SEA,12,17.0,MSP,NW,2008,1230681600
6858076,5.0,2400.0,3,31,-6.0,2124.0,DCA,12,NaN,MIA,AA,2008,1230681600
6858077,7.0,2400.0,3,31,25.0,1937.0,FLL,12,NaN,MSP,NW,2008,1230681600


In [ ]:
df_2008.dtypes
# int64 means LongType
# int32 means IntegerType

In [ ]:
# function if you would like to change a type (we won't as of now)
#df_2008['Timestamp'] = df_2008['Timestamp'].astype('long')

In [13]:
# Drop the columns that we do not need to have for Data_streaming:
df_2008_stream = df_2008.drop(columns=['DayOfWeek', 'Year', 'DayofMonth', 'Month', 'DepTime', 'NASDelay', 'Origin',
                                      'UniqueCarrier'])

In [ ]:
df_2008_stream.head()

In [ ]:
# This is 220 mb, so too large for Github
# df_2008_stream.to_csv("2008_stream.csv", index=False)

In [15]:
# csv file where everything later than 800.0 is dropped
df_2008_stream = df_2008_stream[df_2008_stream['ArrTime'] <= 800.0]
df_2008_stream.to_csv("2008_stream_till_8_2e.csv", index=False)
# df_2008_stream has only 500.000 rows now

In [ ]:
#df_2008_stream['Readable_time'] = pd.to_datetime(df_2008_stream['Timestamp'],unit='s')
# Could add the Readable_time column if wanted

In [ ]:
#df_2008['Timestamp'] = df_2008['Timestamp'].astype('long')
#df_2008_stream.dtypes
#print(df_2008_stream)